<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">





# Procesamiento de lenguaje natural
## Bot con sparCy utilizando un corpus de wikipedia

## 1- instalamos e importamos las librerias necesarias

In [67]:
!pip install spacy==3.3 --quiet
!pip install spacy-stanza==1.0.2 --quiet

In [68]:
import json
import string
import random
import re
import unicodedata
import urllib.request

import nltk
import warnings
warnings.filterwarnings('ignore')

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import stanza
import spacy_stanza

In [69]:
# Descargamos el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")


2022-07-04 18:31:38 INFO: Downloading default packages for language: es (Spanish)...
2022-07-04 18:31:41 INFO: File exists: C:\Users\Lio\stanza_resources\es\default.zip.
2022-07-04 18:31:45 INFO: Finished downloading models and saved to C:\Users\Lio\stanza_resources.
2022-07-04 18:31:46 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-07-04 18:31:46 INFO: Use device: cpu
2022-07-04 18:31:46 INFO: Loading: tokenize
2022-07-04 18:31:46 INFO: Loading: mwt
2022-07-04 18:31:46 INFO: Loading: pos
2022-07-04 18:31:46 INFO: Loading: lemma
2022-07-04 18:31:46 INFO: Loading: depparse
2022-07-04 18:31:47 INFO: Loading: ner
2022-07-04 18:31:48 INFO: Done loading processors!


## 2- Cargamos los datos que vamos a utilizar para nuestro bot
Se consumira los datos del artículo de wikipedia sobre la copa del mundo de Qatar 2022.

Nos quedamos con los párrafos del documento únicamente

URL: https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2022


In [70]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Copa_Mundial_de_F%C3%BAtbol_de_2022')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

### Vemos el texto ya cargado y transformado a minúsculas

In [71]:
article_text

"8 (en 5 ciudades)\nla copa mundial de fútbol de la fifa catar 2022 (en árabe, كأس العالم لكرة القدم قطر 2022) será la xxii edición de la copa mundial de fútbol masculino organizada por la fifa. esta edición se realizará desde el 21 de noviembre al 18 de diciembre de 2022 en catar, que consiguió los derechos de organización el 2 de diciembre de 2010.[1]\u200b\nesta será la tercera vez que la copa del mundo de la fifa se disputará en el continente asiático tras la copa mundial de fútbol de 2002 en corea del sur y japón y el de rusia 2018 (aunque esta última contaba con una única sede en territorio asiático), y la primera que se celebra en asia occidental, dada la ubicación euroasiática de la que gozaba el anterior anfitrión: rusia.[2]\u200b también por primera vez, el torneo tiene lugar en oriente próximo, en un país árabe y de mayoría musulmana, así como el de menor extensión territorial.[3]\u200b\nigualmente será el mundial de mayor tiempo de espera desde 1950 respecto a su edición an

In [72]:
print("Cantidad de caracteres en el artículo es:", len(article_text))

Cantidad de caracteres en el artículo es: 9379


In [73]:
lista_caracteres = list(set(article_text))
print("Lista de caracteres del texto: ",lista_caracteres)

Lista de caracteres del texto:  ['\xa0', 'g', 'q', 't', '6', ';', 'ú', "'", 'ا', '5', 'p', 'c', 'ق', '1', ':', 'ر', 'º', '°', '7', 'n', 'ó', 'é', 'h', '«', 'أ', '[', '3', 'm', 'س', 'f', ')', '2', 'a', 'x', 's', '\n', 'r', '.', ',', ' ', 'y', '%', 'w', '4', '-', 'í', 'k', '»', 'ل', 'd', 'v', 'b', '8', '\u200b', 'o', 'د', ']', 'l', 'ك', 'u', 'e', 'i', 'j', 'ñ', '9', '0', 'á', '(', 'م', 'ة', 'ط', 'ع', 'z']


## 3 - Realizamos en preprocesamiento del texto
- Removemos caracteres especiales
- Removemos acentos
- Quitamos espacios y saltos de línea
- Quitamos las referencias bibliográficas (tags sup)

### Definimos nuestra funcion de limpieza

In [74]:
def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # auitar los suscriptis
    text = re.sub("[\[].*?[\]]", "", text)
    text = text.replace('\u200b', '')
    
    # quitar caracteres especiales
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text


### Aplicamos la limpieza y vemos el texto resultante

In [75]:
text = preprocess_clean_text(article_text)

In [76]:
text

"8 (en 5 ciudades) la copa mundial de futbol de la fifa catar 2022 (en arabe, 2022) sera la xxii edicion de la copa mundial de futbol masculino organizada por la fifa. esta edicion se realizara desde el 21 de noviembre al 18 de diciembre de 2022 en catar, que consiguio los derechos de organizacion el 2 de diciembre de 2010. esta sera la tercera vez que la copa del mundo de la fifa se disputara en el continente asiatico tras la copa mundial de futbol de 2002 en corea del sur y japon y el de rusia 2018 (aunque esta ultima contaba con una unica sede en territorio asiatico), y la primera que se celebra en asia occidental, dada la ubicacion euroasiatica de la que gozaba el anterior anfitrion: rusia. tambien por primera vez, el torneo tiene lugar en oriente proximo, en un pais arabe y de mayoria musulmana, asi como el de menor extension territorial. igualmente sera el mundial de mayor tiempo de espera desde 1950 respecto a su edicion anterior, ya que se desarrollara entre los meses de noviem

In [77]:
print("Cantidad de caracteres en el artículo es:", len(text))

Cantidad de caracteres en el artículo es: 9063


In [78]:
lista_caracteres = list(set(text))
print("Lista de caracteres del texto: ",lista_caracteres)

Lista de caracteres del texto:  ['g', 'q', 't', '6', ';', "'", '5', 'p', 'c', '1', ':', '7', 'n', 'h', '3', 'm', 'f', ')', '2', 'a', 'x', 's', 'r', '.', ',', ' ', 'y', '%', 'w', '4', '-', 'k', 'd', 'v', 'b', '8', 'o', 'l', 'u', 'e', 'i', 'j', '9', '0', '(', 'z']


In [79]:
# Generamos el corpus para ver el listado de documentos generados a partir de nuestro artículo
corpus = nltk.sent_tokenize(text)
corpus

['8 (en 5 ciudades) la copa mundial de futbol de la fifa catar 2022 (en arabe, 2022) sera la xxii edicion de la copa mundial de futbol masculino organizada por la fifa.',
 'esta edicion se realizara desde el 21 de noviembre al 18 de diciembre de 2022 en catar, que consiguio los derechos de organizacion el 2 de diciembre de 2010. esta sera la tercera vez que la copa del mundo de la fifa se disputara en el continente asiatico tras la copa mundial de futbol de 2002 en corea del sur y japon y el de rusia 2018 (aunque esta ultima contaba con una unica sede en territorio asiatico), y la primera que se celebra en asia occidental, dada la ubicacion euroasiatica de la que gozaba el anterior anfitrion: rusia.',
 'tambien por primera vez, el torneo tiene lugar en oriente proximo, en un pais arabe y de mayoria musulmana, asi como el de menor extension territorial.',
 'igualmente sera el mundial de mayor tiempo de espera desde 1950 respecto a su edicion anterior, ya que se desarrollara entre los me

## 4 - Definimos la funcion para el procesamiento completo del texto mediante spaCy
- Tokenization → Lemmatization → Remove stopwords → Remove punctuation
- Quitar símbolos de puntuación

In [80]:
def get_processed_text(text):
    doc = nlp(text)
    
    # Tokenization y lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    
    # Filtrado de stop words
    filtered_sentence =[]
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    # Filtrado de puntuacion
    puntuacion = string.punctuation + "¿¡"
    filtered_sentence = [w for w in filtered_sentence if w not in puntuacion]

    return filtered_sentence

### Visualizamos como hace scapy el procesamiento
SI bien esta función se uitiliza luego dentro del cálculo del vector TF-IDF, ejecutamos el pipeline para ver el resultado obtenido por spacy en el procesamiento del artículo

In [81]:
filtradas = get_processed_text(text)

In [82]:
print("Palabras resultantes del procesamiento:\n",filtradas)

Palabras resultantes del procesamiento:
 ['8', '5', 'ciudad', 'copa', 'mundial', 'futbol', 'fifa', 'catar', '2022', 'arabe', '2022', 'xxii', 'edicion', 'copa', 'mundial', 'futbol', 'masculino', 'organizado', 'fifa', 'edicion', '21', 'noviembre', '18', 'diciembre', '2022', 'catar', 'derecho', 'organizacion', '2', 'diciembre', '2010', 'copa', 'mundo', 'fifa', 'disputar', 'continente', 'asiatico', 'copa', 'mundial', 'futbol', '2002', 'corea', 'sur', 'japon', 'rusia', '2018', 'ultima', 'contar', 'unico', 'sede', 'territorio', 'asiatico', 'celebrar', 'asia', 'occidental', 'ubicacion', 'euroasiatico', 'gozar', 'anfitrion', 'rusia', 'torneo', 'lugar', 'oriente', 'pais', 'arabe', 'mayoria', 'musulmán', 'asar', 'menor', 'extension', 'territorial', 'igualmente', 'mundial', 'tiempo', 'espera', '1950', 'edicion', 'desarrollar', 'mes', 'noviembre', 'diciembre', '2022', 'diferencia', 'mes', 'habitual', 'junio', 'julio', 'forma', 'paralelo', 'copa', 'mundial', 'futbol', 'corto', '1978', 'durar', '28'

## 5- Definimos nuestra función de cálculo del vector de TF-IDF
Vamos a utilziar las stop words en español provistas por spacy

In [83]:
from spacy.lang.es.stop_words import STOP_WORDS as STOP_WORDS_spacy
spacy_stopwords = STOP_WORDS_spacy

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Creamos un vectorizar TFIDF que quite las "stop words" en epañol y 
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=spacy_stopwords)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Lo siento. No puedo entenderte. Prueba con otra consulta"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

## 6 - Ensayamos el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. 

Las sugerencias a ensayar y probadas son:

- mascota
- estadio
- balon
- cancion
- corrupcion
- basquet

In [85]:
pregunta = "¿cual es la mascota del mundial?" 
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿cual es la mascota del mundial?

Respuesta:  la mascota oficial del mundial de catar 2022 es la'eeb que fue presentado el 1 de abril de 2022 en el sorteo para definir los grupos.


In [86]:
pregunta = "¿cual es el nombre del balon del mundial?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿cual es el nombre del balon del mundial?

Respuesta:  al rihla es el nombre oficial del balon para el torneo.


In [87]:
pregunta = "¿donde se realiza el mundial?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿donde se realiza el mundial?

Respuesta:  catar hasta la actualidad confirmo ocho estadios en cinco ciudades para el mundial: al wakrah, doha, rayan, jor y lusail, todas ellas albergaran los 64 partidos de la copa mundial.


In [88]:
pregunta = "¿Cual es la cancion oficial?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Cual es la cancion oficial?

Respuesta:  el 1 de abril de 2022 se lanzo la cancion oficial del mundial llamado hayya hayya (better together) interpretado por el cantante estadounidense trinidad cardona con la participacion del icono del afrobeats davido y aisha.


In [56]:
pregunta = "¿que repercusiones hubo sobre las condiciones laborales en la construccion de los estadios?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

amnistia internacional denuncio practicas de abuso y explotacion contra los trabajadores, especialmente migrantes, en la construccion de los estadios.


In [91]:
pregunta = "¿Que información tenes del partido inaugural?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Que información tenes del partido inaugural?

Respuesta:  la programacion de los partidos fue anunciada por la fifa el 15 de julio de 2020, el comite ejecutivo de la fifa anuncio la hora local de los partidos: el partido inaugural se jugara a las 13:00 y la final se jugara a las 18:00; los partidos correspondientes a las dos primeras fechas de la fase de grupos, a las 13:00, 16:00, 19:00 y 22:00; los partidos de la tercera fecha de la fase de grupos, los octavos y los cuartos de final, a las 18:00 y 22:00; las semifinales a las 22:00; el partido por el tercer puesto y la final a las 18:00. diversas agrupaciones y medios de comunicacion han expresado su preocupacion acerca de la idoneidad de catar para acoger el evento, debido a serios cuestionamientos sobre el respeto de los derechos humanos, particularmente en los casos de las condiciones laborales de los trabajadores y los derechos de la comunidad lgbt, ya que la homosexualidad se llega a condenar con pena de muerte, asi 

In [92]:
pregunta = "¿Es importante el basket en quatar?"
resp = generate_response(pregunta,corpus)
print("Pregunta: ",pregunta)
print("\nRespuesta: ",resp)

Pregunta:  ¿Es importante el basket en quatar?

Respuesta:  Lo siento. No puedo entenderte. Prueba con otra consulta


## 7 - (Opcional) Codigo para pruebas utilizando gradio
A continuación se deja el código que se puede utilizar para testear el bot utilizando gradio

In [ ]:
import sys
!{sys.executable} -m pip install gradio --quiet

In [ ]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

## 8 - Conclusiones

* Se logró generar un bot basándose en un documento HTML de Wikipedia y el vector TD-IDF.
* Se utilizó spacy junto a los diccionarios en español provistos por spacy_stanza. EL idioma español es más complejo para preprocesar que el idioma ingles, debido a los acentos y mayor cantidad de signos de puntuación, además de la complejidad de algunas palabras irregulares del lenguaje.
* Es muy importante conocer el texto y formato del mismo para lograr un buen pre-procesmiento del documento. En lo particular de nuestro caso había tags HTML y caracteres especiales que eran necesarios preprocesar para evitar errores en el procesmiento y lemmatización del texto.
* El texto de wikipedia no es la mejor opción debido al uso de tablas y otros elementos HTML que hacen más complejo el procesamiento. En una prueba aparte se sumaron los datos de las tablas al procesamiento, pero no se obtuvieron resultados mejores en las consultas realizadas. Habia información como los equipos por grupo y paises participantes que si se puedo consultar al agregar las tablas y un preprocesaado de los datos de las mismas, pero haciendo varios ajustes sobre la jerarquía de tags.

Para este último caso se debe filtrar el HTML utilizando el código 
        <code>article_paragraphs = article_html.find_all(['p','table'])</code> y filtrando algunos caracteres extras que aparecen en el código
        
     
